In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

### 0. 测试上传数据
- 文件名: test_upload.csv
- 假设结果正态分布，用random数据。

In [34]:
test = pd.read_csv('season_1/test_0.csv',header=None);test.describe()


,0,2
count,2838.000000,2838.0
mean,33.500000,0.0
std,19.053729,0.0
min,1.000000,0.0
25%,17.000000,0.0
50%,33.500000,0.0
75%,50.000000,0.0
max,66.000000,0.0


In [19]:
test.describe()

,0,2
count,2838.000000,2838.0
mean,33.500000,0.0
std,19.053729,0.0
min,1.000000,0.0
25%,17.000000,0.0
50%,33.500000,0.0
75%,50.000000,0.0
max,66.000000,0.0


In [23]:
import random
test_list = []
for i in range(len(test)):
    test_list.append(abs(random.gauss(0,20)))
random_array = np.array(test_list)

In [35]:
type(random_array[1])

numpy.float64

In [25]:
random_forcast = pd.DataFrame(random_array)

In [31]:
test_result = pd.concat([test[0],test[1],random_forcast[0]],axis=1);test_result.columns=[0,1,2]

In [33]:
test_result.to_csv('test_upload.csv',header=False,index=False)

## 数据获取和清理

### 尝试订单信息获得

In [8]:
order_1 = pd.read_csv('season_1/training_data/order_data/order_data_2016-01-01', delimiter='\t',header=None)
order_1.columns = ["order_id", "driver_id","passenger_id","start_district_hash", "dest_district_hash", "Price","Time"]

In [9]:
order_1.head(5)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,Price,Time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02
3,cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27
4,139d492189ae5a933122c098f63252b3,NaN,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06


In [10]:
order_1.info()
# 可见 总体订单gap 为 (501287- 325577) / 501287 = 0.3505

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501287 entries, 0 to 501286
Data columns (total 7 columns):
order_id               501287 non-null object
driver_id              325577 non-null object
passenger_id           501287 non-null object
start_district_hash    501287 non-null object
dest_district_hash     501287 non-null object
Price                  501287 non-null float64
Time                   501287 non-null object
dtypes: float64(1), object(6)
memory usage: 26.8+ MB


In [33]:
# order_1[order_1.driver_id == NaN]
mask = order_1.isnull();mask
# get the NaN data (gap)
data_no_reply = order_1[mask.driver_id]

In [45]:
# 如何把时间做切片？
a = pd.to_datetime(order_1.Time,unit='s');a.head()

0   2016-01-01 13:37:23
1   2016-01-01 09:47:54
2   2016-01-01 18:24:02
3   2016-01-01 22:13:27
4   2016-01-01 17:00:06
Name: Time, dtype: datetime64[ns]

In [37]:
# 时间目标格式 time slots
test.head()

,0,1,2
0,1,2016-01-22-46,0
1,2,2016-01-22-46,0
2,3,2016-01-22-46,0
3,4,2016-01-22-46,0
4,5,2016-01-22-46,0


In [49]:
# 一一对应 
a[1].hour

9

### 区域的地域属性数据获取

In [7]:
poi_raw = open('season_1/training_data/poi_data/poi_data')

In [9]:
poi_raw.readlines()

['74c1c25f4b283fa74a5514307b0d0278\t1#11:2241\t1#10:249\t24:1245\t25:3652\t20:33449\t22:2324\t23:913\t4:13031\t8:166\t5#4:83\t5#3:3569\t5#2:83\t5#1:4731\t8#2:8798\t8#3:5229\t8#1:664\t8#4:7387\t8#5:83\t1#3:498\t1#2:2822\t1#1:415\t1#7:166\t1#6:83\t1#5:12367\t1#4:249\t1#9:166\t1#8:4316\t14#10:664\t7:6640\t15#7:1411\t15#6:5644\t15#4:249\t15#3:11537\t15#2:4150\t4#8:747\t4#9:2988\t4#6:913\t4#7:1743\t4#4:166\t4#5:4814\t4#2:1328\t4#3:1743\t4#1:664\t16#1:83\t16#2:332\t16#3:332\t16#4:7968\t17#1:249\t16#9:332\t17#3:498\t17#2:12201\t17#5:20999\t17#4:3569\t11#8:36188\t13#8:1660\t11#3:6640\t11#2:7221\t11#1:1992\t11#7:2490\t11#6:18924\t11#5:2822\t11#4:30544\t19#3:27722\t19#2:1245\t19#1:1826\t19#4:581\t14#6:1909\t14#7:1909\t14#2:83\t14#3:1245\t6:8466\t14#8:83\t14#9:83\t4#10:4067\t4#11:3569\t4#12:83\t4#13:7387\t4#14:2656\t4#16:664\t4#17:4150\t4#18:4648\t3#1:3652\t3#3:166\t3#2:166\t11:10956\t15:6059\t14:4399\t17:2988\t16:7636\t19:55859\t6#1:1826\t6#2:664\t6#4:830\t2#12:4648\t2#13:249\t2#10:2490\t2#11:18

In [10]:
poi_1 = pd.read_csv('season_1/training_data/poi_data/poi_data', error_bad_lines=False,delimiter='\t')

b'Skipping line 52: expected 139 fields, saw 142\nSkipping line 53: expected 139 fields, saw 148\nSkipping line 54: expected 139 fields, saw 142\nSkipping line 62: expected 139 fields, saw 152\nSkipping line 63: expected 139 fields, saw 144\n'


In [11]:
poi_1.head()

,74c1c25f4b283fa74a5514307b0d0278,1#11:2241,1#10:249,24:1245,25:3652,20:33449,22:2324,23:913,4:13031,8:166,...,24#1:133713,13#4:19173,2#8:166,2#4:249,2#5:1079,2#6:1245,2#7:1826,2#1:83,2#2:9213,2#3:415
0,08f5b445ec6b29deba62e6fd8b0325a6,20#7:249,20#5:83,2#7:166,20#2:747,20#1:996,16#12:1245,16#10:2158,16#11:415,20#8:249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b7f6f4e2bf237b6cc58f57142bea5c0,4#16:249,24:913,25:332,20:4316,22:415,4:2158,5#4:83,5#3:166,5#1:332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a814069db8d32f0fa6e188f41059c6e1,1#11:498,24:332,25:581,20:5810,22:2407,4:1494,8:83,5#1:581,8#2:996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8316146a6f78cc6d9f113f0390859417,20#7:581,20#5:83,20#4:415,20#2:166,20#1:664,16#12:332,16#10:747,16#11:498,20#8:913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,693a21b16653871bbd455403da5412b4,1#11:249,25:415,20:2905,22:415,4:498,5#3:83,8#2:581,8#3:166,8#4:830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 基准预测